### Getting Satallite Data - Spatio temporal Asset Catalog (STAC)

In [ ]:
import os
import matplotlib.pyplot as plt
import pystac_client
from odc.stac import stac_load, configure_s3_access
import xarray as xr
import rioxarray as rxr

In [ ]:
latitude = 27.163
longitude = 82.608
year = 2023

In [ ]:
catalog = pystac_client.Client.open(
    'https://earth-search.aws.element84.com/v1')

configure_s3_access(
    aws_unsigned=True,
)

km2deg = 1.0 / 111
x, y = (longitude, latitude)
r = 1 * km2deg  # radius in degrees
bbox = (x - r, y - r, x + r, y + r)

search = catalog.search(
    collections=['sentinel-2-c1-l2a'],
    bbox=bbox,
    datetime=f'{year}',
    query={'eo:cloud_cover': {'lt': 20}},
)
items = search.item_collection()

In [ ]:
ds = stac_load(
    items,
    bands=['red', 'green', 'blue', 'nir'],
    resolution=10,
    bbox=bbox,
    chunks={},  # <-- use Dask
    groupby='solar_day',
)

In [ ]:
print(f'DataSet size: {ds.nbytes/1e6:.2f} MB.')

In [ ]:
# items[0].assets.items()

In [ ]:
ds = ds.compute() # lazy vs eager evaluation

In [ ]:
da = ds.red

In [ ]:
# da.isel(time=-1)

In [ ]:
# da.isel(time=-1, x=-1, y=-1).values
# sorted(da.time.values)
# da.sel(time='2023-09-30', method='nearest')
# da.sel(time=slice('2023-01-01', '2023-06-01'))
# da.interp(time='2023-09-30')

In [ ]:
median = ds.median(dim='time')
# stack (red, green, blue, nir) vairables along 'band' dimension
median_da = median.to_array('band') 

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(5,5)
median_da.sel(band=['red', 'green', 'blue']).plot.imshow(
    ax=ax,
    robust=True)
ax.set_title('RGB Visualization')
ax.set_axis_off()
ax.set_aspect('equal')
plt.show()

In [ ]:
# --- exercise: Display the median composite for the month of May.
may_median = ds.groupby('time.month').median(dim='time').to_array('band').sel(month=4)
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(5,5)
may_median.sel(band=['red', 'green', 'blue']).plot.imshow(
    ax=ax,
    robust=True)
ax.set_title('RGB Visualization')
ax.set_axis_off()
ax.set_aspect('equal')
plt.show()

In [ ]:
# writting to geotif with rioxarray
may_median.rio.to_raster('output.tif')

### Visualize Single Scence

In [ ]:
import pandas as pd

timestamp = pd.to_datetime(items[0].properties['datetime']).tz_convert(None)
scene = ds.sel(time=timestamp)
scene = scene.where(scene != 0)
print(f'Scene size: {scene.nbytes/1e6:.2f} MB.')

In [ ]:
# convert raw pixel values to reflectances
scale = 0.0001
offset = -0.1
scene = scene*scale + offset

In [ ]:
scene_da = scene.to_array('band')
print('CRS:', scene_da.rio.crs)
print('Resolution:', scene_da.rio.resolution())

In [ ]:
preview = scene_da.rio.reproject(
    scene_da.rio.crs, resolution=10
)

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(5,5)
preview.sel(band=['red', 'green', 'blue']).plot.imshow(
    ax=ax,
    robust=True)
ax.set_title('RGB Visualization')
ax.set_axis_off()
ax.set_aspect('equal')
plt.show()

In [ ]:
for item in items.properties():
    print(item.properties)